In [91]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
inf = float('inf')
from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard
E = (0,1); W = (0,-1); S = (1,0); N = (-1,0)

In [92]:
file = 'in.txt'
f = open(file).read().split("\n")
lines = [(line) for line in f]
rows, cols = len(lines), len(lines[0])

In [93]:
# grid
grid, neigh = gridneigh(file, to_dict=True, parser=lambda x: list(x))



In [94]:
start = {k for k,v in grid.items() if k[0] == 0 and v == '.'}.pop()
dest = {k for k,v in grid.items() if k[0] == rows-1 and v == '.'}.pop()
start, dest

((0, 1), (140, 139))

In [95]:
def count(nei):
    return sum(grid[tile] != '#' for tile in nei)
splits = {k: count(neigh[k]) for k,v in grid.items() if v != '#' and count(neigh[k]) > 2}

In [84]:
splits = {k for k,v in splits.items()}
splits.add(start)
splits.add(dest)

In [85]:
splitneigh = defaultdict(dict)
for start in splits:
    tocheck = deque([(start, {start})])
    maxx = 0
    found = []
    while tocheck:
        for _ in range(len(tocheck)):
            loc, seen = tocheck.popleft()
            if loc == dest:
                maxx = max(maxx, len(seen))
                found.append(len(seen))
                continue
            # if grid[loc] == '>':
            #     nei = [(loc[0], loc[1]+1)]
            # elif grid[loc] == '<':
            #     nei = [(loc[0], loc[1]-1)]
            # elif grid[loc] == 'v':
            #     nei = [(loc[0]+1, loc[1])]
            # elif grid[loc] == '^':
            #     nei = [(loc[0]-1, loc[1])]
            # else:
            # assert grid[loc] == '.'
            nei = neigh[loc]
            for n in nei:
                if n in grid and grid[n] != '#' and n not in seen:
                    if n in splits:
                        splitneigh[start][n] = len(seen)
                    else:
                        s = seen.copy()
                        s.add(n)
                        tocheck.append((n,s))
                
            


In [86]:
splitneigh[(129, 111)]

{(113, 105): 62, (131, 135): 158, (125, 87): 188}

In [87]:
start = {k for k,v in grid.items() if k[0] == 0 and v == '.'}.pop()
dest = {k for k,v in grid.items() if k[0] == rows-1 and v == '.'}.pop()
start, dest
start, dest

((0, 1), (140, 139))

In [88]:
tocheck = deque([(start, {start},0)])
maxx = 0
found = []
while tocheck:
    for _ in range(len(tocheck)):
        loc, seen, dis = tocheck.popleft()
        # print(loc, seen, dis)
        if loc == dest:
            maxx = max(maxx,dis)
            found.append(seen)
            continue
        nei = splitneigh[loc]
        for n,additional in nei.items():
            if n in grid and grid[n] != '#' and n not in seen:
                s = seen.copy()
                s.add(n)
                tocheck.append((n,s, dis + additional))
                
maxx
            

6898

In [98]:
grid = {k for k,v in grid.items() if v != '#'}
len(grid)

9430

In [99]:
tocheck = deque([(start, {start})])
maxx = 0
found = []
while tocheck:
    for _ in range(len(tocheck)):
        loc, seen = tocheck.popleft()
        if loc == dest:
            maxx = max(maxx, len(seen))
            found.append(len(seen)-1)
            continue
        # if grid[loc] == '>':
        #     nei = [(loc[0], loc[1]+1)]
        # elif grid[loc] == '<':
        #     nei = [(loc[0], loc[1]-1)]
        # elif grid[loc] == 'v':
        #     nei = [(loc[0]+1, loc[1])]
        # elif grid[loc] == '^':
        #     nei = [(loc[0]-1, loc[1])]
        # else:
        #     assert grid[loc] == '.'
        nei = neigh[loc]
        for n in nei:
            if n in grid and n not in seen: #and grid[n] != '#' 
                s = seen.copy()
                s.add(n)
                tocheck.append((n,s))
                
            
maxx

KeyboardInterrupt: 

In [100]:
maxx

2103

In [39]:
seen

95

In [ ]:
2443

In [ ]:
# graph
conn = graphparse("in.txt", parser=lambda line: [x for x in line.split()])
conn = graphparse(
    "in.txt",
    parser=lambda line: [int(x) if x.isnumeric() else x for x in line.split()],
    weights=True,
)

In [6]:
# parse grid with complex numbers
f = open("in.txt").read().split("\n")
lines = [list(line.split()) for line in f]
grid = arr_to_dict(lines, tocomplex=True)
complexneighbors(3+2j)

{(2+2j), (3+1j), (3+3j), (4+2j)}